In [1]:
import numpy as np
import arc
import gurobipy as gp
from gurobipy import GRB
from csp_wizard.node import Node

from time_constraint_shortest_path_solver import TimeConstraintShortestPathSolver
from time_constraint_shortest_path_solver_ver2 import TimeConstraintShortestPathSolverVer2
# Tạo task
from bus_wizard.task import Task
task1 = Task(1,360, 480)
task2 = Task(2,495, 615)
task3 = Task(3,615, 735)
task4 = Task(4,735, 855)
task5 = Task(5,855, 975)
task6 = Task(6,975, 1095)
task7 = Task(7,1095, 1215)
task8 = Task(8,1215, 1335)
tasks = [task1, task2, task3, task4, task5, task6, task7, task8]






In [2]:
T_small = 420 
T_big = 540

In [3]:
# Tạo pattern hợp lệ là 1 ma trận đơn vj
pattern_column = np.identity(8)
pattern_column

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [4]:
# Tạo bài toán restriced master và giải
def solve_restricted_master(pattern_column, presolve = None):
    p = pattern_column.shape[1]
    n = pattern_column.shape[0]
    # Tạo model
    model = gp.Model('restricted_master')
    # Tạo biến  
    x = model.addMVar(shape = p, vtype= GRB.CONTINUOUS, lb =0, name="x")
    model.update()
    # Tạo vector b toàn 1
    b = np.ones(n)
    # Tạo ràng buộc A= pattern_column và Ax  = b
    A = pattern_column
    model.addConstr(A@x == b)
    # Tạo hàm mục tiêu
    model.setObjective(x.sum(), GRB.MINIMIZE)
    #cho presolve là giá trị của presolve
    if presolve is not None:
        x.Start = presolve
    # Giải bài toán
    model.optimize()
    save_solution = x.X
    # return giá trị đối ngẫu của các ràng buộc
    return model.pi, save_solution
    print(model)

In [5]:
pi,s= solve_restricted_master(pattern_column)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 8 rows, 8 columns and 8 nonzeros
Model fingerprint: 0x5a276580
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 8 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  8.000000000e+00


## Sub Problem

In [6]:
#duty arc
def create_odd_node(tasks, position, pi):
    # Tạo node
    id = tasks[position].id
    current_task = tasks[position]
    node = Node(id = id*2-1,
                arcs =[arc.Arc(cost = -pi[position],
                               travel_time = current_task.time_end -current_task.time_start,

                                start_point = id*2-1,
                                  end_point = id*2)])
    return node

In [7]:
node1 = create_odd_node(tasks, 4, pi)
print(node1)

Node: 9, Arcs: [(-1.0,120)]


In [8]:
#connecting arc
def create_even_node(tasks, position, pi, T_small, T_big):
    # Tạo node
    id = tasks[position].id
    current_task = tasks[position]
    #tạo arcs 
    arcs = []
    for next_position in range(position+1,len(tasks)):
        next_id = tasks[next_position].id
        next_task = tasks[next_position]
        if (next_task.time_start >= current_task.time_end
          and next_task.time_start <= current_task.time_end + T_small):
            arcs.append(arc.Arc(cost = -pi[next_position],
                               travel_time = next_task.time_start -current_task.time_end,
                                start_point = id*2,
                                  end_point = next_id*2-1))
    arcs.append(arc.Arc(cost = 0,
                                travel_time = 0,
                                start_point = id*2,
                                  end_point = len(tasks)*2+1,
                                  ))
    node = Node(id = id*2,
                arcs = arcs)
    
    return node

In [9]:
node2 = create_even_node(tasks, 0, pi, T_small, T_big)
node2

Node: 2, Arcs: [(-1.0,15), (-1.0,135), (-1.0,255), (-1.0,375), (0,0)] 

In [10]:
def create_starting_node(tasks, pi, T_small, T_big):
    arcs = []
    for position in range(len(tasks)):
        next_task = tasks[position]
        arcs.append(arc.Arc(cost = -pi[position],
                               travel_time = 0,
                                start_point = 0,
                                  end_point = next_task.id*2-1))
    return Node(id = 0, arcs = arcs)
    return arcs
    

In [11]:
node0 = create_starting_node(tasks, pi, T_small, T_big)
node0

Node: 0, Arcs: [(-1.0,0), (-1.0,0), (-1.0,0), (-1.0,0), (-1.0,0), (-1.0,0), (-1.0,0), (-1.0,0)] 

In [12]:
# tạo list các node
nodes = []
nodes.append(create_starting_node(tasks=tasks, pi=pi, T_small=T_small, T_big=T_big))
for i in range( len(tasks)):
    nodes.append(create_odd_node(tasks, i, pi))
    nodes.append(create_even_node(tasks, i, pi, T_small, T_big))
nodes.append(Node(id = len(tasks)*2+1, arcs = []))
nodes

[Node: 0, Arcs: [(-1.0,0), (-1.0,0), (-1.0,0), (-1.0,0), (-1.0,0), (-1.0,0), (-1.0,0), (-1.0,0)] ,
 Node: 1, Arcs: [(-1.0,120)] ,
 Node: 2, Arcs: [(-1.0,15), (-1.0,135), (-1.0,255), (-1.0,375), (0,0)] ,
 Node: 3, Arcs: [(-1.0,120)] ,
 Node: 4, Arcs: [(-1.0,0), (-1.0,120), (-1.0,240), (-1.0,360), (0,0)] ,
 Node: 5, Arcs: [(-1.0,120)] ,
 Node: 6, Arcs: [(-1.0,0), (-1.0,120), (-1.0,240), (-1.0,360), (0,0)] ,
 Node: 7, Arcs: [(-1.0,120)] ,
 Node: 8, Arcs: [(-1.0,0), (-1.0,120), (-1.0,240), (-1.0,360), (0,0)] ,
 Node: 9, Arcs: [(-1.0,120)] ,
 Node: 10, Arcs: [(-1.0,0), (-1.0,120), (-1.0,240), (0,0)] ,
 Node: 11, Arcs: [(-1.0,120)] ,
 Node: 12, Arcs: [(-1.0,0), (-1.0,120), (0,0)] ,
 Node: 13, Arcs: [(-1.0,120)] ,
 Node: 14, Arcs: [(-1.0,0), (0,0)] ,
 Node: 15, Arcs: [(-1.0,120)] ,
 Node: 16, Arcs: [(0,0)] ,
 Node: 17, Arcs: [] ]

In [13]:
csp = TimeConstraintShortestPathSolver( start_node= 0, end_node= len(tasks)*2+1, nodes= nodes, time_constraint=T_big)
csp.solve()

In [14]:
csp2 = TimeConstraintShortestPathSolverVer2(
     start_node= 0, end_node= len(tasks)*2+1, nodes= nodes, time_constraint=T_small)
csp2.solve()
ans2 = csp2.get_result()
ans2

[0, 1, 2, 3, 4, 5, 6, 17]

In [15]:
ans = csp.get_result()
ans

[0, 1, 2, 3, 4, 5, 6, 7, 8, 17]

In [16]:
# truy vết các vị trí lẻ để ra các task tương ứng
def get_new_pattern_from_path(ans, tasks):
    new_pattern = np.zeros(len(tasks))
    for i in range(len(ans)-1):
        if ans[i]%2 == 1:
            new_pattern[int((ans[i]-1)/2)] = 1
    return new_pattern

In [17]:
new_pattern = get_new_pattern_from_path(ans=ans,tasks=tasks)
new_pattern

array([1., 1., 1., 1., 0., 0., 0., 0.])

In [18]:
#thêm new_pattern vào pattern_column ở cột cuối cùng
pattern_column = np.c_[pattern_column, new_pattern]

In [19]:
new_pi = solve_restricted_master(pattern_column)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 8 rows, 9 columns and 12 nonzeros
Model fingerprint: 0x8eb59c3e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 8 rows and 9 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.000000000e+00


In [20]:
def solve_sub_problem(pi, tasks, T_big, T_small):
        # tạo list các node
    nodes = []
    nodes.append(create_starting_node(tasks=tasks, pi=pi, T_small=T_small, T_big=T_big))
    for i in range( len(tasks)):
        nodes.append(create_odd_node(tasks, i, pi))
        nodes.append(create_even_node(tasks, i, pi, T_small, T_big))
    nodes.append(Node(id = len(tasks)*2+1, arcs = []))
    csp = TimeConstraintShortestPathSolverVer2( start_node= 0, end_node= len(tasks)*2+1, nodes= nodes, time_constraint=T_big)
    csp.solve()
    ans = csp.get_result()
    new_pattern = get_new_pattern_from_path(ans=ans,tasks=tasks)
    return new_pattern
new_pattern
    

    

array([1., 1., 1., 1., 0., 0., 0., 0.])

In [21]:
def solve_final_master_problem(pattern_column):
    # Tạo bài toán restriced master và giải
    p = pattern_column.shape[1]
    n = pattern_column.shape[0]
    # Tạo model
    model = gp.Model('restricted_master')
    # Tạo biến  
    x = model.addMVar(shape = (p), vtype= GRB.BINARY, lb =0, name="x")
    model.update()
    # Tạo vector b toàn 1
    b = np.ones(n)
    # Tạo ràng buộc A= pattern_column và Ax  = b
    A = pattern_column
    model.addConstr(A@x == b)
    # Tạo hàm mục tiêu
    model.setObjective(x.sum(), GRB.MINIMIZE)
    # Giải bài toán
    model.optimize()
    # return giá trị x sao cho x_i=1
    ans =[]
    for i in range(p):
        if x[i].x >= 1-1e-5:
            ans.append(i)
    return ans
    

In [22]:
def column_generation_solve_min_number_drivers(tasks, T_big, T_small, max_iter = 100):
    pattern_columns = np.identity(len(tasks))
    save_solution = np.ones(len(tasks))
    for i in range(max_iter):
        pi,save_solution = solve_restricted_master(pattern_columns, save_solution)
        new_pattern = solve_sub_problem(pi, tasks, T_big, T_small)
        #nếu 1- pi*new_pattern > 0 thì dừng
        if 1 - pi@new_pattern > 1e-5:
            break
        #thêm 1 vào save_solution
        save_solution = np.append(save_solution, 1)
        pattern_columns = np.c_[pattern_columns, new_pattern]
    print("Số patterns ", pattern_columns.shape[1])
    return solve_final_master_problem(pattern_columns), pattern_columns
    

In [23]:
min_drivers_ans, pattern_columns = column_generation_solve_min_number_drivers(tasks, T_big, T_small)
print("Số lượng lái xe tối thiểu cần tìm là: ", len(min_drivers_ans))
min_drivers_ans


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 8 rows, 8 columns and 8 nonzeros
Model fingerprint: 0x6ecd7450
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 8 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  8.000000000e+00
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical p

[8, 9]